 * @ Author: Yohei Ohto
 * @ Create Time: 2024-12-05 18:01:35
 * @ Modified time: 2024-12-05 18:01:46
 * @ Description: 日本の税制の理解と奨学金をもらう人の手取りを計算する　(東京大学、文京区、39歳以下、独身一人暮らし、バイト先が1つでの生活を想定)

In [9]:
# 各種手当に対応していないため、特に交通費はしっかりと抜いてから計算すること　https://edenred.jp/article/employee-benefits/119/#chapter-7

monthly = 18  # 奨学金の月額, 単位は万
annual = monthly * 12 # 奨学金の年額, 単位は万

part_job = 10 * 12 # バイトの年額, 単位は万

all_salary = annual + part_job # あなたの額面での金額 単位は万

all_salary = 240
print("私は一年で",all_salary,"万円稼ぎます!")

私は一年で 240 万円稼ぎます!


In [10]:
# 控除額の計算

def kyuyo_syotoku_kojo_calc(income):
    if income <= 162.5:
        # 給与収入が1,625,000円以下の場合 550,000円
        kojo = 55
    elif income <= 180:
        # 給与収入が1,625,001円から1,800,000円までの場合 収入金額×40％-100,000円
        kojo = income * 0.40 - 10
    elif income <= 360:
        # 給与収入が1,800,001円から3,600,000円までの場合 収入金額×30％+80,000円
        kojo = income * 0.30 + 8
    elif income <= 660:
        # 給与収入が3,600,001円から6,600,000円までの場合 収入金額×20％+440,000円
        kojo = salary * 0.20 + 44
    elif income <= 850:
        # 給与収入が6,600,001円から8,500,000円までの場合 収入金額×10％+1,100,000円
        kojo = income * 0.10 + 110
    else:
        # 給与収入が8,500,001円以上の場合 1,950,000円（上限）
        kojo = 195
    
    return kojo

def kiso_kojo_kuni(income):
    if income <= 2400:
        return 48
    elif income <= 2450:
        return 32
    elif income <= 2500:
        return 16
    else:
        return 0

def kiso_kojo_chiho(income):
    if income <= 2400:
        return 43
    elif income <= 2450:
        return 29
    elif income <= 2500:
        return 15
    else:
        return 0

chihouzei_kiso_kojo = kiso_kojo_chiho(all_salary)
kokuzei_kiso_kojo = kiso_kojo_kuni(all_salary)
kyuyo_syotoku_kojo = kyuyo_syotoku_kojo_calc(all_salary)

print("地方税の基礎控除", chihouzei_kiso_kojo)
print("国税の基礎控除", kokuzei_kiso_kojo)
print("給与所得控除", kyuyo_syotoku_kojo)
# バイト先と奨学金という収入減が複数ある場合には主たる収入減において年末調整を行う義務がある → それぞれで給与所得控除が計算されているため、年末調整時に合算して追徴課税がある

地方税の基礎控除 43
国税の基礎控除 48
給与所得控除 80.0


In [11]:
# 国保 https://www.city.bunkyo.lg.jp/b021/p000424.html
# 年齢的に介護保険を支払う必要がないと想定
kiso_kojo = kiso_kojo_chiho(all_salary)
sante_kiso = all_salary - kyuyo_syotoku_kojo - kiso_kojo # 算定基礎額(所得) = 総所得(給与所得控除が計算済み) - 基礎控除

if sante_kiso < 0:
    sante_kiso = 0 # 基礎控除がマイナスになる場合は0として扱う


syotoku_wari_kiso = sante_kiso * (8.69 / 100) #〔所得割額〕被保険者全員の令和6年度の算定基礎額 × 8.69％
kinto_wari_kiso = 4.9100 * 1 # 〔均等割額〕49,100円 × 被保険者数 (独身想定の為1)
kiso_hoken = syotoku_wari_kiso + kinto_wari_kiso

print("基礎分保険料", kiso_hoken)

syotoku_wari_sien = sante_kiso * (2.80 / 100) #〔所得割額〕被保険者全員の令和6年度の算定基礎額 × 2.80％
kinto_wari_sien = 1.6500 * 1 # 〔均等割額〕16,500円 × 被保険者数(1)
sien_hoken = syotoku_wari_sien + kinto_wari_sien

print("支援金分保険料", sien_hoken)

kokuho = kiso_hoken + sien_hoken
monthly_kokuho = kokuho / 12

print("国保", kokuho, "万円, 月額", monthly_kokuho, "万円")


基礎分保険料 15.0773
支援金分保険料 4.926
国保 20.0033 万円, 月額 1.6669416666666665 万円


In [12]:
# 国民年金 前能でいくらか安くなる、前納時には安くなる口座振替を使用していると仮定する　https://www.nenkin.go.jp/service/kokunen/hokenryo/zenno.html

def kokumin_nenkin_payment(frequency, payment_method):
    """
    国民年金の納付額と割引額を計算する関数（万円単位で返し、年額納付額も計算）。

    Parameters:
        frequency (str): 支払い頻度 ("2年前納", "1年前納", "6カ月前納", "当月末振替", "毎月納付")
        payment_method (str): 支払い方法 ("納付書払い", "クレジットカード払い", "口座振替")

    Returns:
        dict: 納付額（万円単位）、割引額（万円単位）、年額納付額（万円単位）を含む辞書
    """
    # 納付額と割引額の定義（円単位）
    payment_data = {
        "2年前納": {
            "納付書払い": {"amount": 398590, "discount": 15290},
            "クレジットカード払い": {"amount": 398590, "discount": 15290},
            "口座振替": {"amount": 397290, "discount": 16590},
        },
        "1年前納": {
            "納付書払い": {"amount": 200140, "discount": 3620},
            "クレジットカード払い": {"amount": 200140, "discount": 3620},
            "口座振替": {"amount": 199490, "discount": 4270},
        },
        "6カ月前納": {
            "納付書払い": {"amount": 101050, "discount": 830},
            "クレジットカード払い": {"amount": 101050, "discount": 830},
            "口座振替": {"amount": 100720, "discount": 1160},
        },
        "当月末振替": {
            "口座振替": {"amount": 16920, "discount": 60},
        },
        "毎月納付": {
            "納付書払い": {"amount": 16980, "discount": 0},
            "クレジットカード払い": {"amount": 16980, "discount": 0},
            "口座振替": {"amount": 16980, "discount": 0},
        },
    }

    # 各支払い頻度の年額換算用の係数
    frequency_to_multiplier = {
        "2年前納": 0.5,  # 2年間分で割る
        "1年前納": 1,
        "6カ月前納": 2,
        "当月末振替": 12,
        "毎月納付": 12,
    }

    try:
        result = payment_data[frequency][payment_method]
        multiplier = frequency_to_multiplier[frequency]

        # 万円単位で計算（納付額・割引額）
        amount_in_million = result["amount"] / 10000
        discount_in_million = result["discount"] / 10000

        # 年額換算（万円単位）
        annual_amount_in_million = amount_in_million * multiplier
    
        # 月額換算（万円単位）
        monthly_amount_in_million = annual_amount_in_million / 12

        return {
            "支払い頻度": frequency,
            "支払い方法": payment_method,
            "納付額": amount_in_million,
            "割引額": discount_in_million,
            "年額": annual_amount_in_million,
            "月額": monthly_amount_in_million
        }
    except KeyError:
        return {
            "エラー": "指定された支払い頻度または支払い方法が無効です。"
        }


frequency = "当月末振替"
payment_method = "口座振替"
kokumin_nenkin = kokumin_nenkin_payment(frequency, payment_method)

monthly_kokumin_nenkin = kokumin_nenkin["月額"]
annual_kokumin_nenkin = kokumin_nenkin["年額"]

print("国民年金", annual_kokumin_nenkin, "万円, 月額", monthly_kokumin_nenkin, "万円")

国民年金 20.304 万円, 月額 1.692 万円


In [13]:
# 所得税 https://www.nta.go.jp/taxes/shiraberu/taxanswer/shotoku/2260.htm
# 定額減税 https://www.nta.go.jp/users/gensen/teigakugenzei/01.htm

def calculate_income_tax(taxable_income):
    """
    所得税額を計算する関数。
    
    Args:
        taxable_income (float): 課税所得金額（万円単位）
    
    Returns:
        float: 所得税額（万円単位）
    """
    if taxable_income <= 194.9:
        tax_rate = 0.05
        deduction = 0
    elif taxable_income <= 329.9:
        tax_rate = 0.10
        deduction = 9.75
    elif taxable_income <= 694.9:
        tax_rate = 0.20
        deduction = 42.75
    elif taxable_income <= 899.9:
        tax_rate = 0.23
        deduction = 63.6
    elif taxable_income <= 1799.9:
        tax_rate = 0.33
        deduction = 153.6
    elif taxable_income <= 3999.9:
        tax_rate = 0.40
        deduction = 279.6
    else:
        tax_rate = 0.45
        deduction = 479.6

    # 所得税額の計算
    tax = taxable_income * tax_rate - deduction
    return tax

kiso_kojo = kiso_kojo_kuni(all_salary)
kazei_taisyo =  all_salary - kyuyo_syotoku_kojo - kiso_kojo
income_tax = calculate_income_tax(kazei_taisyo)

if all_salary < 2000:
    income_tax -= 3

monthly_income_tax = income_tax / 12
print("所得税", income_tax, "万円, 月額", monthly_income_tax, "万円")

所得税 2.6000000000000005 万円, 月額 0.2166666666666667 万円


In [14]:
# 所得税に付随する税金　復興特別所得税、定額減税
hukko = income_tax * 0.021
monthly_hukko = hukko / 12
print("復興特別所得税", hukko, "万円, 月額", monthly_hukko, "万円")

復興特別所得税 0.05460000000000002 万円, 月額 0.004550000000000001 万円


In [ ]:
# 住民税　https://biz.moneyforward.com/tax_return/basic/49732/#1　https://www.city.bunkyo.lg.jp/b008/p000357.html#sonota　

kiso_kojo = kiso_kojo_chiho(all_salary)
kazei_taisyo_not_tyosei =  all_salary - kyuyo_syotoku_kojo - kiso_kojo - annual_kokumin_nenkin - kokuho

def tyosei_koujo(kazei_taisyo_not_tyosei):
    if kazei_taisyo_not_tyosei <= 200:
        a = abs(kokuzei_kiso_kojo - chihouzei_kiso_kojo)
        b = kazei_taisyo_not_tyosei
        koujo = min(a, b) * 0.05
    else:
        a = abs(kokuzei_kiso_kojo - chihouzei_kiso_kojo)
        b = kazei_taisyo_not_tyosei - 200
        koujo = (a - b) * 0.05
        if koujo < 0:
            koujo = 0
    return koujo

koujo = tyosei_koujo(kazei_taisyo_not_tyosei)
print("調整控除額", koujo)
kazei_taisyo = kazei_taisyo_not_tyosei - koujo
jumin_tax = kazei_taisyo * 0.1 + 0.45 + 0.05
monthly_jumin_tax = jumin_tax / 12
print("住民税", jumin_tax, "万円, 月額", monthly_jumin_tax, "万円")

調整控除額 0.25
住民税 8.1142 万円, 月額 0.6761833333333334 万円


In [16]:
annual_zeikin = jumin_tax + hukko + income_tax + annual_kokumin_nenkin + kokuho
print("あなたは1年で", annual_zeikin, "万円納税しています")

tedori = all_salary - annual_zeikin
print("手取りは", tedori, "万円です")

あなたは1年で 51.0761 万円納税しています
手取りは 188.9239 万円です


In [18]:
# 来年だけ
kokuho = monthly_kokuho * 5 # 7月末に超える → 8月文から国保に入る　https://mynavi-ms.jp/magazine/detail/001339.html
income_tax = income_tax
nenkin = monthly_kokuho * 5 # 8月分から年金を払い始める

kiso_kojo = kiso_kojo_chiho(all_salary)
kazei_taisyo_not_tyosei =  all_salary - kyuyo_syotoku_kojo - kiso_kojo - nenkin - kokuho
koujo = tyosei_koujo(kazei_taisyo_not_tyosei)
print("調整控除額", koujo)
kazei_taisyo = kazei_taisyo_not_tyosei - koujo
jumin_tax = kazei_taisyo * 0.1 + 0.45 + 0.05
monthly_jumin_tax = jumin_tax / 12
print("住民税", jumin_tax, "万円, 月額", monthly_jumin_tax, "万円")

annual_zeikin = jumin_tax + hukko + income_tax + nenkin + kokuho
print("あなたは1年で", annual_zeikin, "万円納税しています")

tedori = all_salary - annual_zeikin
print("手取りは", tedori, "万円です")

調整控除額 0.25
住民税 10.508058333333333 万円, 月額 0.8756715277777777 万円
あなたは1年で 29.832074999999996 万円納税しています
手取りは 210.167925 万円です
